In [1]:
import torch
x= torch.arange(12)
print(x)#向量#tensor 是张量
#访问形状 .shape
print(x.shape)
#访问数量 .numel()
print(x.numel())
#改变形状不改变数量和元素值 .reshape
x = x.reshape(3,4)
print(x )
#张量连接
X = torch.arange(12,dtype = torch.float32).reshape(3,4)
Y = torch.arange(12,dtype = torch.float32).reshape(3,4)
print(X,Y)
a = torch.cat((X,Y),dim=0)#上下叠加
b = torch.cat((X,Y),dim=1)#左右叠加
print(a,b)
print(X == Y)#判断两个张量元素是否相同 


ModuleNotFoundError: No module named 'torch'

In [4]:
import os
import pandas as pd
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n') #列名
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')
#读取文件用到pandas
data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [5]:
#将NaN填充 用到fillna
inputs,outputs= data.iloc[:,0:1] ,data.iloc[:,2] # 提取第1、2列（NumRooms和Alley）
inputs = inputs.fillna(inputs.mean())  # 用均值填充NaN
print(inputs)

   NumRooms
0       3.0
1       2.0
2       4.0
3       3.0


In [6]:
#区别于离散值将NaN为一个类别
inputs = pd.get_dummies(data,dummy_na=True)
print (inputs)


   NumRooms   Price  Alley_Pave  Alley_nan
0       NaN  127500        True      False
1       2.0  106000       False       True
2       4.0  178100       False       True
3       NaN  140000       False       True


In [7]:
#标量的概念 一个元素即为标量
x = torch.Tensor([3.0])
x


tensor([3.])

In [8]:
#标量组成的列表即为向量
y = torch.arange(4)
y
#元素的访问
y[0]

tensor(0)

In [9]:
#张量长度的访问
len(y)

4

In [10]:
#矩阵的创建 先构建再重构
a = torch.arange(12).reshape(3,4)
a

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [11]:
#矩阵的转置 
a.T

tensor([[ 0,  4,  8],
        [ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11]])

In [12]:
#多维矩阵如何创建
B = torch.arange(20).reshape(2,2,5)#从左到右 2为维度 2 为行 5为列 
print(B)

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9]],

        [[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]])


In [13]:
#对元素求和
print(B.sum())#永远都为标量

tensor(190)


In [14]:
#对指定维度求和
B_sum_axis0 = B.sum(axis=0) #把多个矩阵的 “对应位置元素” 相加（即第 0 个矩阵的 (行，列) 和 第 1 个矩阵的 (行，列) 相加），求和后第 0 轴消失，结果维度变为 (3, 4)（3 行 4 列的二维矩阵）
print(B_sum_axis0)
B_sum_axis1 = B.sum(axis=1)#对每个矩阵单独处理，把同一矩阵内的 “每行元素沿列方向相加”（即每个矩阵的所有行求和合并为 1 行），求和后第 1 轴消失，结果维度变为 (2, 4)（2 个矩阵，每个矩阵 1 行 4 列）。
print(B_sum_axis1)
B_sum_axis2 = B.sum(axis=2)#对每个矩阵单独处理，把同一矩阵内的 “每列元素沿行方向相加”（即每个矩阵的每行求和合并为 1 列），求和后第 2 轴消失，结果维度变为 (2, 3)（2 个矩阵，每个矩阵 3 行 1 列）。
print(B_sum_axis2)

tensor([[10, 12, 14, 16, 18],
        [20, 22, 24, 26, 28]])
tensor([[ 5,  7,  9, 11, 13],
        [25, 27, 29, 31, 33]])
tensor([[10, 35],
        [60, 85]])


In [15]:
#求平均值 mean()
Q = torch.arange(12,dtype=torch.float).reshape(2,3,2)#求平均值一定得为浮点数如果是整数无法求平均值
Q.mean()

tensor(5.5000)

In [16]:
#维度求均值
Q_mean_axis0 = Q.mean(axis = 0,keepdim=True)#keepdim 是防止降维的
print(Q_mean_axis0 )


tensor([[[3., 4.],
         [5., 6.],
         [7., 8.]]])


In [17]:
#累加求和
z =Q.cumsum(axis = 0)
print(z)

tensor([[[ 0.,  1.],
         [ 2.,  3.],
         [ 4.,  5.]],

        [[ 6.,  8.],
         [10., 12.],
         [14., 16.]]])


In [18]:
#自动求导
c = torch.arange(4,dtype=float)
c.requires_grad_(True)#用于存储x的梯度
c.grad#默认为None
print(c.grad)
s = 2 * torch.dot(c,c)


None


In [19]:
#正向反向求导思路
import numpy as np
def forward_derivative(x):
    """
    正向求导：计算 y = x² + 3x + 5 在x处的导数 dy/dx
    步骤：
    1. 计算中间变量：a = x², b = 3x, c = 5
    2. 正向计算各中间变量对x的导数，再通过链式法则求dy/dx
    """
    # 第一步：计算函数值和中间变量
    a = x **2  # 中间变量a = x²
    b = 3 * x   # 中间变量b = 3x
    y = a + b + 5  # 最终输出y
    # 第二步：正向求导（按计算顺序求导）
    da_dx = 2 * x  # a对x的导数：d(x²)/dx = 2x
    db_dx = 3      # b对x的导数：d(3x)/dx = 3
    dc_dx = 0      # 常数5对x的导数为0
    dy_dx = da_dx + db_dx + dc_dx  # 链式法则：dy/dx = da/dx + db/dx + dc/dx
    return y, dy_dx
def backward_derivative(x):
    """
    反向求导：从输出y反向计算各变量对x的导数
    步骤：
    1. 先计算所有中间变量和最终输出（和正向计算一致）
    2. 从输出y开始，反向计算每个中间变量的导数（链式法则反向应用）
    """
    # 第一步：正向计算中间变量和输出（和正向求导的第一步一致）
    a = x** 2
    b = 3 * x
    y = a + b + 5
    # 第二步：反向求导（从输出往输入推）
    dy_dy = 1  # 输出对自身的导数为1（起点）
    # y = a + b + 5 → dy/da = 1, dy/db = 1, dy/dc = 1
    dy_da = dy_dy * 1
    dy_db = dy_dy * 1
    dy_dc = dy_dy * 1
    # 反向计算a、b对x的导数，并传递梯度
    da_dx = 2 * x
    db_dx = 3
    dc_dx = 0
    # 最终dy/dx = dy/da * da/dx + dy/db * db/dx + dy/dc * dc/dx
    dy_dx = dy_da * da_dx + dy_db * db_dx + dy_dc * dc_dx
    return y, dy_dx
if __name__ == "__main__":
    # 测试x=2时的导数（理论值：dy/dx=2x+3 → x=2时导数为7）
    x_test = 2
    # 正向求导结果
    y_forward, dy_dx_forward = forward_derivative(x_test)
    print(f"正向求导结果：")
    print(f"x={x_test}时，y={y_forward}，dy/dx={dy_dx_forward}")
    # 反向求导结果
    y_backward, dy_dx_backward = backward_derivative(x_test)
    print(f"\n反向求导结果：")
    print(f"x={x_test}时，y={y_backward}，dy/dx={dy_dx_backward}")

正向求导结果：
x=2时，y=15，dy/dx=7

反向求导结果：
x=2时，y=15，dy/dx=7


In [20]:
#反向传播求导
s.backward()#只能触发一次 如果要第二次触发则得清空
print(c.grad)
c.grad.zero_()

tensor([ 0.,  4.,  8., 12.], dtype=torch.float64)


tensor([0., 0., 0., 0.], dtype=torch.float64)

In [21]:
#python 梯度会累计。所以计算之前得清空之前的值
c.grad.zero_()#清空
y = c.sum()
y.backward()
print(c.grad)

tensor([1., 1., 1., 1.], dtype=torch.float64)


In [22]:
#数据处理演练
# 生成形状为(10, 3)的张量：10行（样本），3列（特征）
# 包含：NaN（缺失值）、极端值（异常值）、重复行、整数类型
torch.manual_seed(0)  # 固定随机种子，结果可复现
raw_data = torch.randn(10, 3)  # 正态分布随机数
# 手动插入缺失值（NaN）
raw_data[1, 0] = float('nan')  # 第2行第1列
raw_data[3, 1] = float('nan')  # 第4行第2列
raw_data[7, :] = float('nan')  # 第8行全部缺失
# 插入异常值（极端值，比如远大于/小于正常范围）
raw_data[2, 2] = 100.0  # 第3行第3列
raw_data[5, 0] = -50.0  # 第6行第1列
# 插入重复行
raw_data[8] = raw_data[0]  # 第9行 = 第1行
raw_data[9] = raw_data[0]  # 第10行 = 第1行
# 转换为整数类型（模拟类型不匹配问题）
raw_data = raw_data.to(torch.int32)

print("===== 原始带问题的张量 =====")
print(raw_data)
print("原始张量形状：", raw_data.shape)
cleaned_data = raw_data.clone()  # 复制张量，避免修改原数据
# 步骤1：类型转换（转为浮点型，方便后续数值计算）
cleaned_data = cleaned_data.to(torch.float32)
print("\n===== 步骤1：类型转换（int32 → float32） =====")
print(cleaned_data.dtype)
# 步骤2：处理缺失值（NaN）
# 方案A：删除全为NaN的行（常用）
# 先判断每行是否有非NaN值，保留非全NaN的行
mask_not_all_nan = ~torch.isnan(cleaned_data).all(dim=1)
cleaned_data = cleaned_data[mask_not_all_nan]
# 方案B：用列均值填充剩余的NaN（常用，替代删除）
# 计算每列的均值（忽略NaN）
col_means = torch.nanmean(cleaned_data, dim=0)
# 遍历每列，填充NaN为对应列的均值
for col in range(cleaned_data.shape[1]):
    nan_mask = torch.isnan(cleaned_data[:, col])
    cleaned_data[nan_mask, col] = col_means[col]
print("\n===== 步骤2：处理缺失值（删除全NaN行 + 均值填充剩余NaN） =====")
print(cleaned_data)
# 步骤3：处理异常值（极端值）
# 方法：用“3σ原则”（正态分布中，99.7%数据在±3σ内）识别异常值，替换为边界值
for col in range(cleaned_data.shape[1]):
    col_data = cleaned_data[:, col]
    mean = torch.mean(col_data)
    std = torch.std(col_data)
    # 计算上下边界
    upper_bound = mean + 3 * std
    lower_bound = mean - 3 * std
    # 替换超过上边界的值为上边界，低于下边界的值为下边界
    cleaned_data[col_data > upper_bound, col] = upper_bound
    cleaned_data[col_data < lower_bound, col] = lower_bound
print("\n===== 步骤3：处理异常值（3σ原则） =====")
print(cleaned_data)
# 步骤4：删除重复行
# 方法：将张量转为numpy（PyTorch无直接去重API），去重后转回张量
# 注意：仅适合小规模张量，大规模数据建议用Pandas先处理
cleaned_np = cleaned_data.numpy()
# 去重：axis=0表示按行去重，return_index=True返回保留行的索引
_, unique_indices = np.unique(cleaned_np, axis=0, return_index=True)
unique_indices = sorted(unique_indices)  # 保持原顺序
cleaned_data = torch.tensor(cleaned_np[unique_indices])
print("\n===== 步骤4：删除重复行 =====")
print(cleaned_data)
# 步骤5：数据标准化（可选，深度学习常用，使数据均值为0、方差为1）
# 计算每列的均值和标准差
col_mean = torch.mean(cleaned_data, dim=0)
col_std = torch.std(cleaned_data, dim=0)
# 标准化：(x - 均值) / 标准差（避免除以0，加极小值）
cleaned_data = (cleaned_data - col_mean) / (col_std + 1e-8)

print("\n===== 步骤5：数据标准化（均值0，方差1） =====")
print(cleaned_data)
print("标准化后每列均值（接近0）：", torch.mean(cleaned_data, dim=0))
print("标准化后每列方差（接近1）：", torch.var(cleaned_data, dim=0))
print("\n===== 最终清洗后的张量 =====")
print(cleaned_data)
print("最终张量形状：", cleaned_data.shape)

===== 原始带问题的张量 =====
tensor([[         -1,          -1,           0],
        [-2147483648,           0,           0],
        [          0,          -2,         100],
        [         -1, -2147483648,           0],
        [          0,           1,           0],
        [        -50,           0,           0],
        [          0,           0,           0],
        [-2147483648, -2147483648, -2147483648],
        [         -1,          -1,           0],
        [         -1,          -1,           0]], dtype=torch.int32)
原始张量形状： torch.Size([10, 3])

===== 步骤1：类型转换（int32 → float32） =====
torch.float32

===== 步骤2：处理缺失值（删除全NaN行 + 均值填充剩余NaN） =====
tensor([[-1.0000e+00, -1.0000e+00,  0.0000e+00],
        [-2.1475e+09,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -2.0000e+00,  1.0000e+02],
        [-1.0000e+00, -2.1475e+09,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
        [-5.0000e+01,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
 